In [1]:
%pip install opencv-python ultralytics numpy

  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (19 kB)
  Using cached ultralytics-8.3.249-py3-none-any.whl.metadata (37 kB)
  Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached matplotlib-3.10.8-cp310-cp310-win_amd64.whl.metadata (52 kB)
  Using cached pillow-12.1.0-cp310-cp310-win_amd64.whl.metadata (9.0 kB)
  Using cached pyyaml-6.0.3-cp310-cp310-win_amd64.whl.metadata (2.4 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached scipy-1.15.3-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached torch-2.9.1-cp310-cp310-win_amd64.whl.metadata (30 kB)
  Using cached torchvision-0.24.1-cp310-cp310-win_amd64.whl.metadata (5.9 kB)
  Using cached contourpy-1.3.2-cp310-cp310-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.61.1-cp310-cp310-win_amd64.whl.metadata (116 kB)
  Using cached kiwisolver-1.4.9-cp310-cp310-win_amd64.whl.me

In [3]:
import cv2
import numpy as np
from ultralytics import YOLO

# =========================
# LOAD MODEL ONCE
# =========================
model = YOLO("yolov8n.pt")

# =========================
# OPEN WEBCAM ONCE
# =========================
cap = cv2.VideoCapture(0)  # change to 1 if external webcam

if not cap.isOpened():
    print("❌ Webcam not accessible")
    exit()

# =========================
# VIDEO SAVE (OPTIONAL)
# =========================
save_video = True  # set True if you want to save output

if save_video:
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(
        "output_multivision.avi",
        cv2.VideoWriter_fourcc(*"XVID"),
        20,
        (w, h)
    )

# =========================
# EFFECT FUNCTIONS
# =========================
def normal(frame):
    return frame

def grayscale(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)

def thermal(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return cv2.applyColorMap(gray, cv2.COLORMAP_JET)

def night_vision(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    eq = cv2.equalizeHist(gray)
    green = np.zeros_like(frame)
    green[:, :, 1] = eq
    return green

# =========================
# MODE MAP
# =========================
modes = {
    "1": ("Normal", normal),
    "2": ("Grayscale", grayscale),
    "3": ("Thermal", thermal),
    "4": ("Night Vision", night_vision),
}

current_mode = "1"

print("""
==============================
YOLO MULTI-VISION SYSTEM
1 - Normal
2 - Grayscale
3 - Thermal
4 - Night Vision
Q - Quit
==============================
""")

# =========================
# MAIN LOOP (ONLY ONE)
# =========================
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Apply selected effect
    effect_name, effect_fn = modes[current_mode]
    processed = effect_fn(frame)

    # YOLO expects RGB
    rgb = cv2.cvtColor(processed, cv2.COLOR_BGR2RGB)
    results = model(rgb)

    # Draw boxes
    output = results[0].plot()
    output = cv2.cvtColor(output, cv2.COLOR_RGB2BGR)

    # Display mode text
    cv2.putText(
        output,
        f"Mode: {effect_name}",
        (20, 40),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 255, 0),
        2
    )

    cv2.imshow("YOLO Multi-Vision", output)

    if save_video:
        out.write(output)

    key = cv2.waitKey(1) & 0xFF

    if key == ord("q"):
        break
    elif chr(key) in modes:
        current_mode = chr(key)

# =========================
# CLEANUP
# =========================
cap.release()
if save_video:
    out.release()
cv2.destroyAllWindows()



YOLO MULTI-VISION SYSTEM
1 - Normal
2 - Grayscale
3 - Thermal
4 - Night Vision
Q - Quit


0: 480x640 1 person, 1 bottle, 193.7ms
Speed: 40.2ms preprocess, 193.7ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 bottle, 182.0ms
Speed: 5.7ms preprocess, 182.0ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 179.2ms
Speed: 9.8ms preprocess, 179.2ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 171.8ms
Speed: 4.4ms preprocess, 171.8ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 155.2ms
Speed: 3.4ms preprocess, 155.2ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 bottle, 145.0ms
Speed: 3.2ms preprocess, 145.0ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 161.6ms
Speed: 3.7ms preprocess, 161.6ms inference, 1.5ms postprocess per image at shape 